In [1]:
import re
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

from tqdm import tqdm

from os import listdir
from os.path import isfile, join

In [2]:
mypath = '../medelement_clin_prot_rk/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [3]:
delete_sections = ['ATTENTION', 'ATTACHMENTS', 'LISTOFREFERENCES', 'CLASSIFICATION']
delete_sections = set(delete_sections)

words = ['мм', 'рт', 'ст', 'г', 'м', 'мл', 'кг', 'мг', 'см', 'ч', 'е', 'х'] 
set_stopWords = stopwords.words("russian") + words
set_stopWords = set(set_stopWords)

In [4]:
def deleteStopWords(text):
    stop = ['др. греч.', 'греч.', 'лат.', 'др.', 'и т. д', 'и т.д', 'т. е.', 'т.е.', 'т.н.', 'т. н.', 'т.п.', 'т. п.',
                'т. д', 'т.д', 'мм. рт. ст.', 'мм рт. ст.', 'мм.рт.ст', 'МЕДПОИСК.РУ', 'Контактная форма ниже', 'ºС']
    for i in stop:
        text = text.replace(i, '')
    return text

In [5]:
def clearTable(text):
    text = deleteStopWords(text)
    text = text.replace('\xad', '')
    text = text.replace('\n\n', '.')
    text = text.replace('..', '.')
    text = text.replace(':', ' ')
    text = text.replace(';', '.')
    text = text.replace('?', '.')
    text = text.replace('!', '.')
    text = re.sub(r'\[[\d+]*\]', '', text)
    text = re.sub(r'\([\d+]*\)', '', text)
    text = re.sub('\d+', ' ', text)
    text = re.sub('[^\w.]', ' ', text)
    
    final_s = ''
    for sen in text.split('.'):
        sen = sen.strip()
        tmp_t = ''
        for tok in sen.split(' '):
            tok = tok.strip()
            if(tok.lower() not in set_stopWords):
                tmp_t += tok + ' '
        if(len(tmp_t.split()) < 2):
            continue
        final_s += tmp_t + '.'
    return final_s

In [6]:
def clearHtml(fileName):
    text = ''
    with open(mypath + fileName) as file:
        text = file.read()
    soup = BeautifulSoup(text, 'lxml')
    
    soup.find('section', {'class': 'page-information disease-information'}).decompose()
    soup.find('section', {'data-index': '0'}).decompose()
    
    for name in delete_sections:
        for sec in soup.find_all('section', {'data-section-name': name}):
            sec.decompose()
        for sec in soup.find_all('section', {'data-field-name': name}):
            sec.decompose()
    
    final_text = ''
    for i in soup.find_all(lambda tag: tag.name=='table'):
        tmp = clearTable(i.text)
        final_text += tmp
        i.decompose()
    
    tmp = clearTable(soup.text)
    final_text += tmp
    
    return final_text

In [7]:
allText = ''
for file in tqdm(onlyfiles):
    fileText = clearHtml(file)
    allText += fileText

100%|██████████| 1023/1023 [02:45<00:00,  6.17it/s]


In [8]:
len(allText)

23561331

In [9]:
# with open('medelement-rk.txt', 'w+') as f:
#     f.write(allText)